# library

In [35]:
import os
import sys
import re
import pickle
import random
import subprocess
import time
import threading
import shutil
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, wait, ALL_COMPLETED, as_completed
from datetime import datetime, timedelta
from multiprocessing import Process, Pool

import numpy as np
import pandas as pd
import anndata as ad
import h5py
# import Bio
# from Bio import motifs
import pysam
import pyranges
import pybedtools
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import sklearn
from sklearn import preprocessing
import scipy
from scipy import io
import scanpy as sc
from sklearn.cluster import KMeans
from adjustText import adjust_text
# import episcanpy
import ruamel.yaml
yaml = ruamel.yaml.YAML(typ="safe")
yaml.default_flow_style = False


import SCRIPT
from SCRIPT.utilities import utils
from SCRIPT.utilities.utils import print_log, safe_makedirs, excute_info

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
# warnings.simplefilter(action='ignore', category=subprocess.)

plt.rcParams.update({
    'figure.figsize': [8.0, 8.0],
    'font.size' : 15,
    'font.family': 'Arial',
    'font.style' : 'normal',
    'font.weight':'bold',
    'figure.titleweight': 'bold',
    'axes.labelsize': 14 ,
    'axes.titleweight': 'normal',
    'axes.labelweight': 'bold',
    'axes.spines.right': False,
    'axes.spines.top': False,
})

N = 256
vals = np.ones((N, 4))
vals[:, 0] = np.linspace(220/256, 34/256, N)
vals[:, 1] = np.linspace(220/256, 7/256, N)
vals[:, 2] = np.linspace(220/256, 141/256, N)
regulation_cmp = mpl.colors.ListedColormap(vals)

In [20]:
def preprocess(inputadata, min_cells=10, min_genes=200, max_genes=7000, title=''):
    adata = inputadata.copy()
    sc.pp.filter_cells(adata, min_genes=min_genes)
    sc.pp.filter_genes(adata, min_cells=min_cells)
    sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)
    adata = adata[adata.obs.n_genes_by_counts < max_genes, :]
    sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts'], jitter=0.4, multi_panel=True)
    sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')
    sc.tl.pca(adata, svd_solver='arpack')
    sc.pl.pca_variance_ratio(adata, log=True)
    sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
    sc.tl.umap(adata)
    sc.tl.louvain(adata)
    sc.pl.umap(adata, color=['louvain'], legend_fontsize=15, title=title)
    return adata

In [44]:
H3K27ac = sc.read_10x_h5('example/histone/data/matrix/H3K27ac_bin_count.h5', gex_only=False)
# H3K4me1 = sc.read_10x_h5('example/histone/data/H3K4me1_bin_count.h5', gex_only=False)
# H3K4me3 = sc.read_10x_h5('example/histone/data/H3K4me3_bin_count.h5', gex_only=False)

In [ ]:
# H3K27ac = sc.read_10x_mtx('example/histone/data/fastq/H3K27ac_H3K4me1/H3K27ac_merge_filtered/', gex_only=False)
# H3K4me1 = sc.read_10x_mtx('example/histone/data/fastq/H3K27ac_H3K4me1/H3K4me1_merge_filtered/', gex_only=False)
# H3K4me3 = sc.read_10x_mtx('example/histone/data/fastq/H3K4me3/H3K4me3_merge_filter_filtered/', gex_only=False)
# ATAC = sc.read_10x_mtx('example/histone/data/fastq/paired-seq/Access_merge_filter_filtered/', gex_only=False)

In [ ]:
H3K27ac_processed = preprocess(H3K27ac, min_cells=0, min_genes=0, max_genes=70000)

In [ ]:
H3K4me1_processed = preprocess(H3K4me1, min_cells=50, min_genes=500, max_genes=6000)

In [ ]:
H3K4me1_processed

In [ ]:
sc.pl.umap(H3K4me1_processed, color='n_genes_by_counts')

In [ ]:
H3K4me1

In [ ]:
H3K4me3_processed = preprocess(H3K4me3, min_cells=0, min_genes=0, max_genes=10000)

In [ ]:
ATAC_processed = preprocess(ATAC, min_cells=0, min_genes=0, max_genes=10000)

In [ ]:
min_peaks = 'auto'
min_cells = 'auto'
max_peaks = 'auto'

In [ ]:
featurstore_to_picklerix = sc.read_10x_h5('example/PBMC/data/ATAC/filtered_mtx/PBMC_granulocyte_sorted_10k_filtered_peak_count.h5', gex_only=False)

In [ ]:
min_peaks = 'auto'
min_cells = 'auto'
max_peaks = 'auto'
sc.pp.calculate_qc_metrics(feature_matrix, percent_top=None, log1p=False, inplace=True)
feature_mean = feature_matrix.obs.n_genes_by_counts.mean()
feature_std = feature_matrix.obs.n_genes_by_counts.std()

if min_cells == 'auto':
    min_cells = int(0.005 * feature_matrix.n_obs)
else:
    min_cells = int(min_cells)
sc.pp.filter_genes(feature_matrix, min_cells=min_cells)

if min_peaks == 'auto':
    min_peaks = 500 if feature_mean-3*feature_std < 500 else feature_mean-3*feature_std
else:
    min_peaks = int(min_peaks)
sc.pp.filter_cells(feature_matrix, min_genes=min_peaks)  # filter by n_genes_by_counts

if max_peaks == 'auto':
    max_peaks = feature_mean+3*feature_std
else:
    max_peaks = int(max_peaks)
feature_matrix = feature_matrix[feature_matrix.obs.n_genes_by_counts < max_peaks, :]

In [ ]:
feature_matrix.var

In [ ]:
feature_matrix

In [ ]:
sns.displot(H3K4me3.obs.n_genes_by_counts)

In [ ]:
H3K27ac_processed = preprocess(H3K27ac, min_cells=50, min_genes=500, max_genes=7000, title='H3K27ac')

In [ ]:
H3K4me1_processed = preprocess(H3K4me1, min_cells=50, min_genes=500, max_genes=15000, title='H3K4me1')

In [ ]:
H3K4me3_processed = preprocess(H3K4me3, min_cells=10, min_genes=200, max_genes=3000, title='H3K4me3')

In [ ]:
sc.pp.calculate_qc_metrics(H3K27ac, percent_top=None, log1p=False, inplace=True)

sc.pl.violin(H3K27ac, ['n_genes_by_counts', 'total_counts'], jitter=0.4, multi_panel=True)

sc.pl.scatter(H3K27ac, x='total_counts', y='n_genes_by_counts')

H3K27ac = H3K27ac[H3K27ac.obs.n_genes_by_counts < 7000, :]

H3K27ac

# sc.pp.scale(H3K27ac, max_value=10)
sc.tl.pca(H3K27ac, svd_solver='arpack')
sc.pl.pca_variance_ratio(H3K27ac, log=True)
sc.pp.neighbors(H3K27ac, n_neighbors=10, n_pcs=40)
sc.tl.umap(H3K27ac)

sc.tl.louvain(H3K27ac)

sc.pl.umap(H3K27ac, color=['louvain'], legend_fontsize=15, title='H3K27ac')

In [24]:
metadata = pd.read_csv('example/histone/data/matrix/meta.tsv', sep='\t', index_col=0)

In [25]:
metadata

,Tissue,Rep,Target,Total_RNA_Reads,Mapped_RNA_Reads,Uniquely_Mapped_RNA_Reads,UMI_RNA,nGene_RNA,Total_DNA_Reads,Mapped_DNA_Reads,Uniquely_Mapped_DNA_Reads,nFragments_DNA,Membership,Annotation
Cell_ID,,,,,,,,,,,,,,
01:02:16:08,FC,2,H3K27ac,19425,18643,18211,5818,999,5426,5284,5093,1878,3,L5
01:02:19:07,FC,1,H3K27ac,6358,5891,5700,2043,557,2281,2225,2102,793,15,OPC
01:02:21:07,FC,1,H3K27ac,4726,4500,4399,1545,434,7735,7484,7066,2687,16,Oligo_MOL
01:02:21:09,FC,3,H3K27ac,15582,14707,14363,4851,1002,17282,16824,15981,6374,3,L5
01:02:21:10,HC,1,H3K27ac,6385,5980,5861,2035,519,4987,4841,4634,1809,11,DG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56:96:79:12,HC,2,ATAC,6511,5211,4847,3424,726,22814,21542,17774,6529,18,Astro_Myoc
56:96:93:04,HC,1,ATAC,56864,51186,49159,32865,3503,25773,23399,20056,6497,12,CGE
56:96:94:01,FC,1,ATAC,7444,5227,4181,3245,778,8004,7542,6501,2310,12,CGE


# enhance singal

In [32]:
from sklearn.neighbors import KDTree
from sklearn.neighbors import BallTree

def find_nearest_cells(q_point, tree, n_neighbor):
    _, ind = tree.query(q_point, k=n_neighbor+1)
    return ind[0][1:]

def cal_neighbor_cell_peak_mat(sub_mat, input_mat, tree, pc_table, impute_n, start_idx, i):
    end_index = start_idx + sub_mat.shape[0]
    k = 0
    for idx in range(start_idx, end_index):
        nearest_bc_idx = find_nearest_cells(np.reshape(pc_table[idx,:], (1,-1)), tree, n_neighbor=impute_n)
#         scipy.sparse.csr_matrix(input_mat[nearest_bc_idx,:].sum(0))
        sub_mat[k,:] = input_mat[nearest_bc_idx,:].sum(0)
        k += 1
    return sub_mat

def cal_neighbor_cell_peak_mat_batch(input_mat, impute_n=5, KD_leafsize=80, nPC = 50, n_cores=8):
    '''
    input_mat:
    a csr sparse matrix, which can be get by adata.X
    
    '''
    print_log("Building KD tree...")
    pc_table = sc.tl.pca(input_mat, n_comps=50, svd_solver='arpack')
    tree = BallTree(pc_table, KD_leafsize)
    print_log("Calculating neighbors, divide into {n} chunks...".format(n=n_cores))
    cell_number = input_mat.shape[0]
    index_split = [i for i in range(0,cell_number,int(cell_number/n_cores))] + [cell_number]
#     input_table_split = np.array_split(input_mat_dense, n_cores)
    input_mat_lil = input_mat.tolil()
    input_mat_split = [input_mat_lil[index_split[i]:index_split[i+1],:] for i in range(index_split.__len__()-1)]
    args = [[sub_mat, input_mat, tree, pc_table, impute_n, index_split[i], i] for (i, sub_mat) in enumerate(input_mat_split)]
#     print(args)
    with Pool(n_cores) as p:
        result = p.starmap(cal_neighbor_cell_peak_mat, args)
    cell_peak_csr = scipy.sparse.vstack(result).tocsr()
    print_log('Finished!')
    return cell_peak_csr

In [2]:
input_mat_adata = sc.read_10x_h5('example/histone/data/matrix/ATAC_bin_count.h5', gex_only=False)
# input_mat = input_mat_adata.X.copy()

In [ ]:
imputed_csr = cal_neighbor_cell_peak_mat_batch(input_mat, impute_n=5, KD_leafsize=80, nPC = 50, n_cores=2)
utils.store_to_pickle(imputed_csr, 'imputed.csr.pk')

In [3]:
imputed_csr = utils.read_pickle('imputed.csr.pk')

# impute singal

In [6]:
def generate_peak_list(cells, input_mat, peak_confidence=1):
    cell_above_cutoff_index = sc.pp.filter_genes(
        input_mat[cells, :], min_cells=peak_confidence, inplace=False)[0]
    peaks = input_mat.var_names[cell_above_cutoff_index].to_list()
    return peaks


def generate_beds(file_path, cells, input_mat, peak_confidence=1):
    peaks = generate_peak_list(cells, input_mat, peak_confidence)
    cell_barcode = os.path.basename(file_path)[:-4]  # remove .bed
    if peaks.__len__() == 0:
        print_log('Warning: No peaks in {bed_path}, skip generation'.format(bed_path=file_path[:-4]))
    else:
        peaks = pd.DataFrame([p.rsplit("_", 2) for p in peaks])
        peaks.to_csv(file_path, sep="\t", header=None, index=None)
        cmd = 'sort --buffer-size 2G -k1,1 -k2,2n -k3,3n {bed_path} | bgzip -c > {bed_path}.gz\n'.format(bed_path=file_path)
        cmd += 'rm {bed_path}'.format(bed_path=file_path)
        subprocess.run(cmd, shell=True, check=True)
    return [cell_barcode, peaks.__len__()]


def generate_beds_by_matrix(cell_feature_adata, beds_path, peaks_number_path, n_cores):
    safe_makedirs(beds_path)
    # total_cnt = adata.obs.index.__len__()
    executor = ThreadPoolExecutor(max_workers=n_cores)
    all_task = []
    for cell in cell_feature_adata.obs.index:
        # neighbor_cells = find_nearest_cells(cell, coor_table, n_neighbor, step)
        # map_dict[cell] = neighbor_cells
        all_task.append(executor.submit(generate_beds, beds_path + "/" + str(cell) + ".bed", cell, cell_feature_adata))
    wait(all_task, return_when=ALL_COMPLETED)
    pd.DataFrame([_.result() for _ in as_completed(all_task)]).to_csv(peaks_number_path, header=None, index=None, sep='\t')
    return

In [4]:
input_mat_adata.X = imputed_csr

In [ ]:
generate_beds_by_matrix(input_mat_adata, 'example/histone/SCRIPT/imputation/imputed_beds/', 'example/histone/SCRIPT/imputation/imputed_beds_peaks_number.txt', 16)

In [5]:
def search_seqpare_factor(bed_path, result_path, index_path, factor):
    cmd = 'seqpare "{index_path}/{factor}*.bed.gz" "{bed_path}" -m 1 -o {result_path}\n'.format(
        index_path=index_path, result_path=result_path, bed_path=bed_path, factor=factor)
    subprocess.run(cmd, shell=True, check=True)


def search_seqpare_factor_batch(bed_folder, result_folder, index_path, factor, n_cores=8, tp=''):
    print_log('Start searching beds from {tp} index ...'.format(tp=tp))
    safe_makedirs(result_folder)
    beds = os.listdir(bed_folder)
    args = []
    for bed in beds:
        barcodes = bed[:-7]  # remove suffix '.bed.gz'
        args.append((os.path.join(bed_folder, bed),
                     os.path.join(result_folder, barcodes + '.txt'),
                     index_path,
                     factor))
    with Pool(n_cores) as p:
        p.starmap(search_seqpare_factor, args)
    print_log('Finished searching beds from {tp} index ...'.format(tp=tp))

In [54]:
search_seqpare_factor_batch('example/histone/SCRIPT/imputation/imputed_beds/', 
                            'example/histone/SCRIPT/imputation/imputed_results_H3K4me1/', 
                            '/fs/home/dongxin/Projects/SCRIPT/indices/mouse/hm_chip_qc_1_20k',
                           'H3K4me1',
                           16)

INFO 2021-08-30 09:48:09 Start searching beds from  index ...
INFO 2021-08-30 10:09:27 Finished searching beds from  index ...


In [56]:
search_seqpare_factor_batch('example/histone/SCRIPT/imputation/imputed_beds/', 
                            'example/histone/SCRIPT/imputation/imputed_results_H3K27ac/', 
                            '/fs/home/dongxin/Projects/SCRIPT/indices/mouse/hm_chip_qc_1_20k',
                           'H3K27ac',
                           16)

INFO 2021-08-30 10:29:02 Start searching beds from  index ...
INFO 2021-08-30 11:10:33 Finished searching beds from  index ...


In [6]:
def get_factor_affinity(input_mat, bed_file_path, reference, factor, ccre_number):
    peaks = input_mat.var_names.to_list()
    peaks_number = peaks.__len__()
    ref_number = pd.read_csv(reference + '/peaks_number.txt', sep='\t', index_col=0, header=None)
    factor_idx = [i for i in ref_number.index if i.startswith(factor)]
    ref_number = ref_number.loc[factor_idx,:]
    
    peaks = pd.DataFrame([p.rsplit("_", 2) for p in peaks])
    peaks.to_csv(bed_file_path, sep="\t", header=None, index=None)
    cmd = 'sort --buffer-size 2G -k1,1 -k2,2n -k3,3n {bed_path} | bgzip -c > {bed_path}.gz\n'.format(bed_path=bed_file_path)
    cmd += 'rm {bed_path}'.format(bed_path=bed_file_path)
    subprocess.run(cmd, shell=True, check=True)

    search_seqpare_factor(bed_file_path + '.gz', bed_file_path[0:-4] + '.txt', reference, factor)
    all_peak_result = read_seqpare_result([bed_file_path[0:-4] + '.txt'])
    affinity = all_peak_result.iloc[:,0]/(ref_number[1]*peaks_number/ccre_number)
    return affinity

def read_seqpare_result(files):
    for i in range(len(files)):
        giggle_result = os.path.basename(files[i])
        cell_bc = giggle_result[:-4]  # remove suffix '.txt'
        dtframe = pd.read_csv(files[i], sep="\t", index_col=5)
        if i == 0:
            dtframe = dtframe.loc[:, ['teo']].copy()
            dataset_cell_score_df = dtframe.rename(columns={'teo': cell_bc})
        else:
            dataset_cell_score_df[cell_bc] = dtframe.loc[:, "teo"]
    dataset_cell_score_df.index = [i.rsplit('/', 1)[1][:-7] for i in dataset_cell_score_df.index]  # remove suffix '.bed.gz'
    return dataset_cell_score_df

def read_seqpare_result_batch(path, n_cores=8, tp=''):
    print_log("Reading searching results, using {n} cores...".format(n=n_cores))
    file_list = os.listdir(path)
    result_split = np.array_split(file_list, n_cores)
    args = [[[os.path.join(path, j) for j in list_chunk]] for list_chunk in result_split]
    with Pool(n_cores) as p:
        result = p.starmap(read_seqpare_result, args)
    dataset_cell_score_df = pd.concat([i for i in result], axis=1)
    print_log("Finished reading {tp} index search result!".format(tp=tp))
    return dataset_cell_score_df

In [8]:
H3K4me1_affinity = get_factor_affinity(input_mat_adata, 'example/histone/SCRIPT/imputation/all_beds.bed', '/fs/home/dongxin/Projects/SCRIPT/indices/mouse/hm_chip_qc_1_20k', 'H3K4me1', 339815)

In [7]:
H3K27ac_affinity = get_factor_affinity(input_mat_adata, 'example/histone/SCRIPT/imputation/all_beds.bed', '/fs/home/dongxin/Projects/SCRIPT/indices/mouse/hm_chip_qc_1_20k', 'H3K27ac', 339815)

In [9]:
H3K4me1_enrich = read_seqpare_result_batch('example/histone/SCRIPT/imputation/imputed_results_H3K4me1/')

INFO 2021-09-07 11:27:41 Reading searching results, using 8 cores...
INFO 2021-09-07 11:27:47 Finished reading  index search result!


In [10]:
H3K27ac_enrich = read_seqpare_result_batch('example/histone/SCRIPT/imputation/imputed_results_H3K27ac/')

INFO 2021-09-07 11:27:47 Reading searching results, using 8 cores...
INFO 2021-09-07 11:28:11 Finished reading  index search result!


In [11]:
def cal_peak_factor_norm(ref_peak_number_path, peaks_number_path, ccre_number, affinity, factor):
    ref_peak_number = pd.read_csv(ref_peak_number_path, sep='\t', header=None, index_col=0)
    factor_idx = [i for i in ref_peak_number.index if i.startswith(factor)]
    ref_peak_number = ref_peak_number.loc[factor_idx,:]
    data_peak_number = pd.read_csv(peaks_number_path, sep='\t', header=None, index_col=0)
    peak_norm_hyper = (ref_peak_number.dot(data_peak_number.T)/ccre_number)
    peak_norm_hyper = (peak_norm_hyper.T * affinity).T
    return peak_norm_hyper


def cal_score(dataset_mbm_overlap_df, dataset_bg_peak_norm_df):
    intersect_frame = dataset_mbm_overlap_df.copy()
    peak_hyper_frame = dataset_bg_peak_norm_df.copy().reindex(index=intersect_frame.index, columns=intersect_frame.columns)
    fg_dataset_cell_raw_score_df = np.log2((intersect_frame+1)/(peak_hyper_frame+1))
    return fg_dataset_cell_raw_score_df

In [12]:
H3K4me1_hyper_bg = cal_peak_factor_norm('/fs/home/dongxin/Projects/SCRIPT/indices/mouse/hm_chip_qc_1_20k/peaks_number.txt', 'example/histone/SCRIPT/imputation/imputed_beds_peaks_number.txt', 339815, H3K4me1_affinity, 'H3K4me1')

In [13]:
H3K27ac_hyper_bg = cal_peak_factor_norm('/fs/home/dongxin/Projects/SCRIPT/indices/mouse/hm_chip_qc_1_20k/peaks_number.txt', 'example/histone/SCRIPT/imputation/imputed_beds_peaks_number.txt', 339815, H3K27ac_affinity, 'H3K27ac')

In [42]:
# ref_peak_number[1][ref_peak_number.index[ref_peak_number[1] > 5000]]

In [14]:
H3K4me1_score = cal_score(H3K4me1_enrich, H3K4me1_hyper_bg)

In [15]:
ref_peak_number = pd.read_csv('/fs/home/dongxin/Projects/SCRIPT/indices/mouse/hm_chip_qc_1_20k/peaks_number.txt', sep='\t', header=None, index_col=0)
factor_idx = [i for i in ref_peak_number.index if i.startswith('H3K4me1')]
ref_peak_number = ref_peak_number.loc[factor_idx,:]
H3K4me1_score = H3K4me1_score.loc[ref_peak_number.index[ref_peak_number[1] > 5000],:].copy()

In [ ]:
H3K4me1_score

In [16]:
H3K27ac_score = cal_score(H3K27ac_enrich, H3K27ac_hyper_bg)

In [17]:
ref_peak_number = pd.read_csv('/fs/home/dongxin/Projects/SCRIPT/indices/mouse/hm_chip_qc_1_20k/peaks_number.txt', sep='\t', header=None, index_col=0)
factor_idx = [i for i in ref_peak_number.index if i.startswith('H3K27ac')]
ref_peak_number = ref_peak_number.loc[factor_idx,:]
H3K27ac_score = H3K27ac_score.loc[ref_peak_number.index[ref_peak_number[1] > 5000],:].copy()

In [18]:
@excute_info('Getting the best reference for each cell.')
def get_factor_source(table):
    ret_table = table.copy()
    # map factor by id "_"
    factor_index_list = []
    for i in ret_table.index:
        factor_name = i.split("_")
        factor_index_list.append(factor_name[0])
    ret_table.loc[:, "Factor"] = factor_index_list
    max_index = ret_table.groupby("Factor").idxmax()
    return max_index

In [19]:
H3K4me1_source = get_factor_source(H3K4me1_score)

INFO 2021-09-07 11:28:46 Getting the best reference for each cell.


In [20]:
H3K4me1_source.loc['H3K4me1',:].unique().__len__()

90

In [21]:
H3K27ac_source = get_factor_source(H3K27ac_score)

INFO 2021-09-07 11:28:49 Getting the best reference for each cell.


In [28]:
metadata['H3K27ac'] = H3K27ac_source.T
metadata['H3K4me1'] = H3K4me1_source.T

In [34]:
metadata.to_csv('example/histone/data/matrix/meta_with_source.tsv', sep='\t', index=True)

In [52]:
ref_path = '/fs/home/dongxin/Projects/SCRIPT/indices/mouse/hm_chip_qc_1_20k/'
chip_bed_list = [pybedtools.BedTool(os.path.join(ref_path, i + '.bed.gz')) for i in H3K27ac_source.iloc[0,:].unique()]
chip_bed = chip_bed_list[0].cat(*chip_bed_list[1:])
print(chip_bed.__len__())

113783


In [53]:
data_bed = pybedtools.BedTool('\n'.join(['\t'.join(p.rsplit('_', maxsplit=2)) for p in input_mat_adata.var_names]), from_string=True)
intersect_bed = data_bed.intersect(chip_bed, u=True)
imputed_chip_peak = str(intersect_bed).replace('\t','_').split('\n')[0:-1]

In [54]:
chip_cell_peak = input_mat_adata[:,imputed_chip_peak].copy()

In [55]:
chip_cell_peak

AnnData object with n_obs × n_vars = 14095 × 189973
    var: 'gene_ids', 'feature_types', 'genome'

In [56]:
chip_cell_peak_df = chip_cell_peak.to_df()

In [32]:
for i in H3K4me1_source.iloc[0,:].unique():
    cellbc = H3K4me1_source.columns[H3K4me1_source.iloc[0,:] == i]
    tmp_dataset_bed = pybedtools.BedTool(os.path.join(ref_path, i + '.bed.gz'))
#     print(str(intersect_bed.intersect(tmp_dataset_bed)).replace('\t','_').split('\n')[0:-1].__len__())
    exclude_chip_peak = str(intersect_bed.intersect(tmp_dataset_bed, v=True)).replace('\t','_').split('\n')[0:-1]
#     ovlp = 441896 - exclude_chip_peak.__len__()
#     print(i)
#     print(H3K4me1_source.columns[H3K4me1_source.iloc[0,:] == i].__len__())
#     print(ovlp)
    chip_cell_peak_df.loc[cellbc,exclude_chip_peak] = 0

In [57]:
for i in H3K27ac_source.iloc[0,:].unique():
    cellbc = H3K27ac_source.columns[H3K27ac_source.iloc[0,:] == i]
    tmp_dataset_bed = pybedtools.BedTool(os.path.join(ref_path, i + '.bed.gz'))
#     print(str(intersect_bed.intersect(tmp_dataset_bed)).replace('\t','_').split('\n')[0:-1].__len__())
    exclude_chip_peak = str(intersect_bed.intersect(tmp_dataset_bed, v=True)).replace('\t','_').split('\n')[0:-1]
#     ovlp = 441896 - exclude_chip_peak.__len__()
#     print(i)
#     print(H3K4me1_source.columns[H3K4me1_source.iloc[0,:] == i].__len__())
#     print(ovlp)
    chip_cell_peak_df.loc[cellbc,exclude_chip_peak] = 0

In [ ]:
chip_cell_peak_df.sum(1).sort_values()

In [58]:
chip_cell_peak = sc.AnnData(chip_cell_peak_df)

In [59]:
chip_cell_peak.X = scipy.sparse.csr.csr_matrix(chip_cell_peak.X)

In [60]:
chip_cell_peak.X[chip_cell_peak.X>1] = 1

In [36]:
def write_to_mtx(data, path):
    if not os.path.exists(path):
        os.makedirs(path)
    pd.DataFrame(data.var.index).to_csv(os.path.join(path, "genes.tsv" ), sep = "\t", index=False, header=False)
    pd.DataFrame(data.obs.index).to_csv(os.path.join(path, "barcodes.tsv"), sep = "\t", index=False, header=False)
    data.obs.to_csv(os.path.join(path, "metadata.tsv"), sep = "\t", index=False, header=False)
    io.mmwrite(os.path.join(path, "matrix.mtx"), data.X)

In [37]:
write_to_mtx(chip_cell_peak, 'example/histone/SCRIPT/imputation/mtx_imputed_H3K4me1/')

In [61]:
write_to_mtx(chip_cell_peak, 'example/histone/SCRIPT/imputation/mtx_imputed_H3K27ac/')

In [129]:
chip_cell_peak.X.sum(1).min()

17.0

In [130]:
chip_cell_peak

AnnData object with n_obs × n_vars = 14095 × 441896

In [235]:
cellbc = H3K4me1_source.columns[H3K4me1_source.iloc[0,:] == i]

In [192]:
ref_path = '/fs/home/dongxin/Projects/SCRIPT/indices/mouse/hm_chip_qc_1_20k/'
chip_bed_list = [pybedtools.BedTool(os.path.join(ref_path, i + '.bed.gz')) for i in H3K27ac_source.iloc[0,:].unique()]

In [ ]:
chip_bed = chip_bed_list[0].cat(*chip_bed_list[1:])
print(chip_bed.__len__())
data_bed = pybedtools.BedTool('\n'.join(['\t'.join(p.rsplit('_', maxsplit=2)) for p in input_mat_adata.var_names]), from_string=True)
chip_peak = str(data_bed.intersect(chip_bed, u=True)).replace('\t','_').split('\n')[0:-1]

In [ ]:
chip_peak

In [196]:
chip_cell_peak.X[chip_cell_peak.X>1] = 1

In [197]:
chip_cell_peak

AnnData object with n_obs × n_vars = 14095 × 190367
    var: 'gene_ids', 'feature_types', 'genome'

In [198]:
input_mat_adata.obs_names

Index(['51:02:02:12', '51:02:14:06', '51:02:14:09', '51:02:14:12',
       '51:02:15:07', '51:02:15:10', '51:02:24:01', '51:02:24:08',
       '51:02:27:09', '51:02:27:12',
       ...
       '56:96:25:10', '56:96:57:05', '56:96:75:02', '56:96:75:11',
       '56:96:77:09', '56:96:79:12', '56:96:93:04', '56:96:94:01',
       '56:96:94:08', '56:96:94:09'],
      dtype='object', length=14095)

In [199]:
input_mat_adata.X

<14095x2443832 sparse matrix of type '<class 'numpy.float32'>'
	with 150294688 stored elements in Compressed Sparse Row format>

In [ ]:
# chip_bed = pybedtools.BedTool('/mnt/Storage/home/dongxin/Files/cistrome/human/human_factor/46129_sort_peaks.narrowPeak.bed')
tmp_bed = pybedtools.BedTool('\n'.join(['\t'.join(p.rsplit('_', maxsplit=2)) for p in ret_cell_peak.columns]), from_string=True) # p[::-1].replace('_','\t',2)[::-1]
chip_peak = str(tmp_bed.intersect(chip_bed, u=True)).replace('\t','_').split('\n')[0:-1]

chip_peak.__len__()

# chip_cell_peak = np.sqrt(chip_cell_peak)

chip_cell_peak = ret_cell_peak[chip_peak].copy()

cells_list = chip_cell_peak.index

peaks_list = chip_cell_peak.columns.tolist()
# peaks_list = [p.encode() for p in peaks_list]
peaks_info= []
for ipeak, peak in enumerate(peaks_list):
    peaks_tmp = peak.rsplit("_", maxsplit=2)
    peaks_info.append([peaks_tmp[0][3:], (int(peaks_tmp[1]) + int(peaks_tmp[2])) / 2.0, 0, ipeak])

genes_peaks_score_dok = RP_Simple(peaks_info, genes_info, 10000)

genes_peaks_score_csr = genes_peaks_score_dok.tocsr()
genes_cells_score_csr = genes_peaks_score_csr.dot(scipy.sparse.csr_matrix(chip_cell_peak).T)

score_cells_dict = {}
score_cells_sum_dict = {}

for igene, gene in enumerate(genes_list):
    score_cells_dict[gene] = igene
    score_cells_sum_dict[gene] = genes_cells_score_csr[igene, :].sum()

score_cells_dict_dedup = {}
score_cells_dict_max = {}
genes = list(set([i.split("@")[0] for i in genes_list]))
for gene in genes:
    score_cells_dict_max[gene] = float("-inf")

for gene in genes_list:
    symbol = gene.split("@")[0]
    if score_cells_sum_dict[gene] > score_cells_dict_max[symbol]:
        score_cells_dict_dedup[symbol] = score_cells_dict[gene]
        score_cells_dict_max[symbol] = score_cells_sum_dict[gene]
gene_symbol = sorted(score_cells_dict_dedup.keys())
matrix_row = []
for gene in gene_symbol:
    matrix_row.append(score_cells_dict_dedup[gene])

score_cells_matrix = genes_cells_score_csr[matrix_row, :]

RP_table = pd.DataFrame(score_cells_matrix.todense(), index=gene_symbol, columns=cells_list)

In [13]:
n_cores = 2
KD_leafsize = 80

In [177]:
index_split = [i for i in range(0,cell_number,int(cell_number/n_cores))] + [cell_number]

In [178]:
input_mat

<14095x2443832 sparse matrix of type '<class 'numpy.int64'>'
	with 31302243 stored elements in Compressed Sparse Row format>

In [24]:
input_mat_dense = input_mat.to_df()

In [14]:
# pc_table = sc.tl.pca(input_mat, n_comps=50, svd_solver='arpack')
tree = BallTree(pc_table, KD_leafsize)

In [18]:
q_point = np.reshape(pc_table[1,:], (1,-1))

In [20]:
nearest_bc_idx = find_nearest_cells(q_point, tree, n_neighbor=5)

In [27]:
scipy.sparse. 

matrix([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

<1x2443832 sparse matrix of type '<class 'numpy.float32'>'
	with 2885 stored elements in Compressed Sparse Row format>

In [170]:
a = input_mat.astype(int)

In [146]:
ret = input_mat_lil.copy()

In [156]:
nearest_bc_idx = find_nearest_cells(q_point, kd_tree, n_neighbor=5)
# nearest_bc = bc_list[nearest_bc_idx]

In [157]:
nearest_bc_idx

array([ 7101,  7000, 13471, 11007, 13312])

In [149]:
ret[0,:] = input_mat_csr[nearest_bc_idx,:].sum(0)

In [29]:
%%timeit
ret[0,:] = input_mat_csr[nearest_bc_idx,:].sum(0)

144 ms ± 155 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [117]:
scipy.sparse.vstack([ret[0,:], ret[1,:]])

<2x2443832 sparse matrix of type '<class 'numpy.float32'>'
	with 5926 stored elements in COOrdinate format>

In [49]:
ret.shape

<14095x2443832 sparse matrix of type '<class 'numpy.float32'>'
	with 31306602 stored elements in List of Lists format>

In [99]:
index_split = [i for i in range(0,14095,int(np.ceil(14095/5)))] + [14095]

In [100]:
index_split

[0, 2819, 5638, 8457, 11276, 14095]

In [101]:
# index_split = [i for i in range(0,cell_number,int(cell_number/n_cores))]
# #     input_table_split = np.array_split(input_mat_dense, n_cores)
input_mat_split = [input_mat.X[index_split[i]:index_split[i+1],:] for i in range(index_split.__len__()-1)]

In [102]:
input_mat_split

[<2819x2443832 sparse matrix of type '<class 'numpy.float32'>'
 	with 4971229 stored elements in Compressed Sparse Row format>,
 <2819x2443832 sparse matrix of type '<class 'numpy.float32'>'
 	with 5748312 stored elements in Compressed Sparse Row format>,
 <2819x2443832 sparse matrix of type '<class 'numpy.float32'>'
 	with 7549531 stored elements in Compressed Sparse Row format>,
 <2819x2443832 sparse matrix of type '<class 'numpy.float32'>'
 	with 7281010 stored elements in Compressed Sparse Row format>,
 <2819x2443832 sparse matrix of type '<class 'numpy.float32'>'
 	with 5752161 stored elements in Compressed Sparse Row format>]

In [62]:
ret[14080:14760,:]

<15x2443832 sparse matrix of type '<class 'numpy.float32'>'
	with 29912 stored elements in List of Lists format>

In [16]:
%%timeit
input_mat_dense.loc[nearest_bc,:].sum()

3.21 s ± 42.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
nearest_bc_idx

array([ 7101,  7000, 13471, 11007, 13312])

In [ ]:
input_mat_dense.loc[nearest_bc,:].sum()

In [ ]:
print_log("Calculating neighbors, divide into {n} chunks...".format(n=n_cores))
input_table_split = np.array_split(input_mat_dense, n_cores)
args = [[table, input_mat_dense, tree, bc_list, impute_n, i] for (i, table) in enumerate(input_table_split)]
with Pool(n_cores) as p:
    result = p.starmap(cal_neighbor_cell_peak_mat, args)
cell_peak = pd.concat([i for i in result])
print_log('Finished!')
#     return sp.sparse.csr_matrix(cell_peak)

In [ ]:
ATAC_impute = cal_neighbor_cell_peak_mat_batch(ATAC, 5, 80, 16)
utils.store_to_pickle(ATAC_impute, 'example/histone/data/matrix/ATAC_impute.pk')

INFO 2021-08-17 11:34:29 Building KD tree...
INFO 2021-08-17 11:35:41 Calculating enrichment, divide into 16 chunks...


In [11]:
sc.pp.calculate_qc_metrics(ATAC, inplace=True)

In [ ]:
ATAC_pp = preprocess(ATAC, min_cells=50, min_genes=200, max_genes=7000, title='')

In [ ]:
sc.tl.pca(ATAC, n_comps=50, svd_solver='arpack')

In [25]:
ATAC_pp

AnnData object with n_obs × n_vars = 14095 × 36153
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'n_genes', 'louvain'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'
    uns: 'pca', 'neighbors', 'umap', 'louvain', 'louvain_colors'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

In [23]:
ATAC

AnnData object with n_obs × n_vars = 14095 × 2443832
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'